In [1]:
import os
import pandas as pd
import geopandas as gpd
from math import ceil
from shapely import wkt
from pyproj import Transformer
from shapely.geometry import box
from tqdm.notebook import tqdm

In [2]:
# Read OBIS occurence data into dataframe
raw_df = pd.read_csv('data\OBIS_anemone_occurrences_slim.tsv', sep='\t')
raw_df['geometry'] = raw_df['geometry'].apply(wkt.loads)
raw_gdf = gpd.GeoDataFrame(raw_df, geometry='geometry', crs=4326)

# Transform occurence data to Behrmann projection
behrmann_occurences_gdf = raw_gdf.to_crs("ESRI:54017")

In [3]:
# Read GBIF occurence data into dataframe
raw_df = pd.read_csv('data\GBIF_species_ID_complete.csv')

### Data Cleaning and Pre-processing

In [4]:
# Remove null coordinates
raw_df = raw_df.dropna(subset=['decimalLongitude', 'decimalLatitude'])

# Remove points at origin (coordinate 0, 0)
raw_df = raw_df[(raw_df[['decimalLongitude','decimalLatitude']] != 0).all(axis=1)]

In [5]:
# Read into geodataframe
raw_gdf = gpd.GeoDataFrame(
    raw_df, 
    geometry=gpd.points_from_xy(raw_df['decimalLongitude'], raw_df['decimalLatitude']),
    crs='EPSG:4326'
)

# Transform to Behrmann projection
behrmann_raw_gdf = raw_gdf.to_crs('ESRI:54017')

In [6]:
# Read Natural Lands ocean map
ocean_gdf = gpd.read_file('data\\ne_10m_ocean.shp', crs='ESRI:4326')

# Transform to Behrmann projection
ocean_gdf = ocean_gdf.to_crs('ESRI:54017')

# Remove occurence points 10km or more inland
ocean_10km_buffer = ocean_gdf.buffer(10000)
occurences_gdf = behrmann_raw_gdf.clip(ocean_10km_buffer)


### Generate fishnet grid

In [7]:
def create_fishnet(grid_size, bbox, projection):
    # Initialize the transformer to convert from geographic (longitude, latitude) to Behrmann projection
    transformer = Transformer.from_crs("EPSG:4326", projection, always_xy=True)
    
    # Transform the bounding box coordinates to the projection
    min_x, min_y = transformer.transform(bbox[0], bbox[1])
    max_x, max_y = transformer.transform(bbox[2], bbox[3])
    
    # Calculate number of cells needed in the x and y directions
    x_cells = ceil(int((max_x - min_x) / grid_size))
    y_cells = ceil(int((max_y - min_y) / grid_size))
    
    # Generate polygons for each cell in the grid
    polygons = []
    tile_ids = []
    for i in range(x_cells):
        for j in range(y_cells):
            # Coordinates of the lower left corner of the grid cell
            x1 = min_x + i * grid_size
            y1 = min_y + j * grid_size
            # Create a polygon for the grid cell
            poly = box(x1, y1, x1 + grid_size, y1 + grid_size)
            tile_id = f"{i}_{j}"
            polygons.append(poly)
            tile_ids.append(tile_id)
    
    # Create a GeoDataFrame from the polygons
    grid = gpd.GeoDataFrame({'tile_id': tile_ids, 'geometry': polygons}, crs=projection)
    return grid

In [8]:
# Define bounding box in geographic coordinates [min_lon, min_lat, max_lon, max_lat]
bbox = [-180, -80, 180, 80]  # Adjust latitude as per the usability in cylindrical projections

# Grid size in meters (e.g., 1000 km)
grid_size = 800000

# Create the fishnet grid
grid = create_fishnet(grid_size, bbox, "ESRI:54017")

In [9]:
# Join occurence data to grid
occurences_joined_gdf = occurences_gdf.sjoin(grid, how="inner", predicate="within")

In [10]:
cells = occurences_joined_gdf['tile_id'].unique().tolist()

### Create subgrids and transform occurence data to incidence

In [11]:
def create_subgrid(geom):
    min_x, min_y, max_x, max_y = geom.bounds
    x_cells = int((max_x - min_x) / 2000)
    y_cells = int((max_y - min_y) / 2000)

    polygons = []
    tile_ids = []
    for i in range(x_cells):
        for j in range(y_cells):
            # Coordinates of the lower left corner of the grid cell
            x1 = min_x + i * 2000
            y1 = min_y + j * 2000
            # Create a polygon for the grid cell
            poly = box(x1, y1, x1 + 2000, y1 + 2000)
            tile_id = f"{i}_{j}"
            polygons.append(poly)
            tile_ids.append(tile_id)

    return gpd.GeoDataFrame({'subgrid_id': tile_ids, 'geometry': polygons}, crs='ESRI:54017')

def species_counts(df):
    return df['species'].value_counts().to_list()

In [12]:
def gbif2inext(occurences: gpd.GeoDataFrame, grid: gpd.GeoDataFrame, output_dir: str):

    # Join occurence data to grid
    joined = occurences.sjoin(grid, how="inner", predicate="within")

    # List of cells with occurences
    cells = joined['tile_id'].unique().tolist()

    # Maximum number of occurences in a cell
    max_species_count = 0

    # Number of sampling units
    grid_size = int(grid['geometry'].iloc[0].bounds[3] - grid['geometry'].iloc[0].bounds[1])
    num_sampling_units = int((grid_size / 2000) ** 2)

    for cell in tqdm(cells):
        # Get occurences in the cell
        cell_occurences = joined[joined['tile_id'] == cell].drop('index_right', axis=1)

        # Create subgrid
        geom = grid[grid['tile_id'] == cell]['geometry'].item()
        subgrid = create_subgrid(geom)

        # Join occurences to subgrid
        joined_subgrid = cell_occurences.sjoin(subgrid, how='inner', predicate='within').drop('index_right', axis=1)

        # Identify assemblage
        assemblage = species_counts(joined_subgrid)

        # Calc species count and reset max_species_count if neededdd
        if len(assemblage) > max_species_count:
            max_species_count = len(assemblage)

        export = pd.DataFrame(assemblage, columns=[cell])
        sampling_unit_row = pd.DataFrame([num_sampling_units], columns=export.columns)
        export = pd.concat([sampling_unit_row, export]).reset_index(drop=True)
        export.to_csv(f'{output_dir}\\{cell}.csv', index=False)

    return max_species_count

In [13]:
max_species_count = gbif2inext(occurences_gdf, grid, 'inext_input')

  0%|          | 0/356 [00:00<?, ?it/s]

In [14]:
# Extend every file to max_species_count

def extend_df(dir, max_species_count):
    fps = [os.path.join(dir, f) for f in os.listdir(dir)]
    for fp in fps:
        df = pd.read_csv(fp)
        diff = max_species_count - len(df)
        if diff > 0:
            addl_rows = pd.DataFrame([[0]*len(df.columns)]*diff, columns=df.columns)
            df = pd.concat([df, addl_rows]).reset_index(drop=True)
        df.to_csv(fp, index=False)

    return

In [15]:
extend_df('inext_input', max_species_count)